In [4]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import plotly.express as px

load_dotenv()

True

In [5]:
# define the endpoint
api_key = os.environ.get('WEATHER_MAP_KEY')
city = input("Select a city:")
endpoint = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"

In [6]:
# send the request
response = requests.get(endpoint)
response.status_code

200

In [7]:
# get the data from the response
data = response.json()

In [8]:
temperatures = []
temperature_feel_like = []
weather_descriptions = []
wind_speeds = []
dates = []

for prediction in data['list']:
    temperatures.append(prediction['main']['temp'])
    temperature_feel_like.append(prediction['main']['feels_like'])
    weather_descriptions.append(prediction['weather'][0]['description'])
    wind_speeds.append(prediction['wind']['speed'])
    dates.append(prediction['dt_txt'])

In [34]:
# create the Data Frame
df_weather = pd.DataFrame({
    'date': dates,
    'temperature': temperatures,
    'temperature_feels_like': temperature_feel_like,
    'description': weather_descriptions,
    'wind_speed': wind_speeds,
    'city_name' : data['city']['name'],
    'long': data['city']['coord']['lon'],
    'lat': data['city']['coord']['lat'],
    'country': 'NLD',
    'continent': 'Europe'
})

In [30]:
df_weather.head()

,date,temperature,temperature_feels_like,description,wind_speed,city_name,long,lat
0,2024-10-17 12:00:00,17.76,17.78,scattered clouds,4.28,Amsterdam,4.8897,52.374
1,2024-10-17 15:00:00,17.99,18.04,light rain,2.87,Amsterdam,4.8897,52.374
2,2024-10-17 18:00:00,15.96,15.99,broken clouds,1.09,Amsterdam,4.8897,52.374
3,2024-10-17 21:00:00,15.46,15.44,overcast clouds,1.48,Amsterdam,4.8897,52.374
4,2024-10-18 00:00:00,14.78,14.72,overcast clouds,1.18,Amsterdam,4.8897,52.374


In [11]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    40 non-null     object 
 1   temperature             40 non-null     float64
 2   temperature_feels_like  40 non-null     float64
 3   description             40 non-null     object 
 4   wind_speed              40 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.7+ KB


In [12]:
#pd.to_datetime(df_weather['date'], format="%Y-%m-%d %H:%M:%S")
df_weather['date'] = pd.to_datetime(df_weather['date'])

df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    40 non-null     datetime64[ns]
 1   temperature             40 non-null     float64       
 2   temperature_feels_like  40 non-null     float64       
 3   description             40 non-null     object        
 4   wind_speed              40 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 1.7+ KB


In [95]:
with pd.ExcelWriter('export.xlsx') as writer:
    df_weather.to_excel(writer, sheet_name="weather_predictions")

In [ ]:
#!pip install openpyxl

In [15]:
fig = px.bar(df_weather, x='date', y='temperature')
fig.show()

In [22]:
fig = px.density_heatmap(df_weather, x="wind_speed", y="temperature", z="temperature_feels_like", histfunc="avg")
fig.show()

In [25]:
data['city']

{'id': 2759794,
 'name': 'Amsterdam',
 'coord': {'lat': 52.374, 'lon': 4.8897},
 'country': 'NL',
 'population': 2122311,
 'timezone': 7200,
 'sunrise': 1729145373,
 'sunset': 1729183294}

In [45]:
import plotly.express as px

# Create the map visualization
fig = px.scatter_geo(df_weather, locations="country", color="continent",
                     hover_name="temperature", size="wind_speed")

# Update the geographic layout to focus on the Netherlands
fig.update_geos(
    center={"lat": 52.1326, "lon": 5.2913},  # Latitude and longitude of the Netherlands
    scope="europe",  # Scope is still Europe
    lataxis_range=[50, 54],  # Latitude range to zoom in
    lonaxis_range=[3, 8],    # Longitude range to zoom in
)

fig.show()